# Script work for Generating Charting Outputs

In [22]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide
alt.themes.enable("fivethirtyeight")

import clean_data
import _dla_utils

In [8]:
def make_charts(filterbycol, filterbyvalue):     
    df= pd.read_parquet("dla_df.parquet")
    df = (df>>filter(_[filterbycol]==[filterbyvalue]))
    
    #subsetting the dat
    df_years = _dla_utils.count_all_years(df)
    df_top = _dla_utils.find_top(df)
    
    
    ##charting df_years
    
    
    
    
    ##charting df_tops 
    
    values = (df_top['variable'].unique())
    
    for value in values:
        #need to change the folder directory for saving functions
        chart = _dla_utils.basic_bar_chart((df_top>>filter(_.variable==values)), 'value', 'count')
        return chart

In [20]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "total_requested": "Total Requested",
              "fed_requested":"Fed Requested",
              "ac_requested": "Advance Construction Requested",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word


In [14]:
def basic_bar_chart_test(df, x_col, y_col):
    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col), sort=('-y')),
                 y=alt.Y(y_col, title=labeling(y_col)),
                 #column = "payment:N",
                 color = alt.Color(y_col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(y_col)))
                                  ))
             .properties( 
                          title=f"Highest {labeling(x_col)} by {labeling(y_col)}")
    )

    chart=styleguide.preset_chart_config(chart)
    #chart.save(f"./chart_outputs/bar_{x_col}_by_{y_col}.png")
    
    return chart

In [9]:
df_top = _dla_utils.find_top(df)

In [34]:
values = sorted(df_top['variable'].unique())

In [36]:
values

['dist',
 'mpo',
 'prefix',
 'prepared_y',
 'primary_agency_name',
 'project_location',
 'seq',
 'status_comment',
 'type_of_work']

In [38]:
for value in values:
        #need to change the folder directory for saving functions
    basic_bar_chart_test((df_top>>filter(_.variable==value)), 'value', 'count')

    chart.display()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)